## Testboards

In [ ]:
#Startboard
board1 = [[9, 9],
         [[0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0]]]
#Phase 1
board2 = [[7, 6],
         [[0, 0, 0, 0, 2, 0, 0, 0],
          [0, 1, 2, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 1, 2, 0, 0]]]
#Phase 2, keine Mühle
board3 = [[0, 0],
         [[1, 0, 1, 2, 0, 1, 0, 0],
          [2, 0, 1, 0, 0, 0, 1, 0],
          [0, 2, 0, 0, 1, 2, 0, 0]]]
#Phase 2, Mühle
board4 = [[0, 0],
         [[1, 1, 1, 2, 0, 1, 0, 0],
          [2, 0, 1, 0, 0, 0, 1, 0],
          [0, 2, 0, 0, 1, 2, 0, 0]]]
#Phase 3 für Spieler 1
board5 = [[0, 0],
         [[0, 0, 1, 2, 0, 1, 0, 0],
          [2, 0, 0, 0, 0, 0, 0, 0],
          [0, 2, 0, 0, 1, 2, 0, 0]]]

## Spielphasen

Prüft in welcher Phase sich der Spieler befindet

In [ ]:
def get_player_phase(pieces, player):
    [remaining, board] = pieces
    if remaining[player-1] >= 1: # Es wurden noch nicht alle 9 Steine gesetzt
            return 1
    elif count_player_pieces(pieces, player) == 3: # nur noch drei Steine auf dem Spielfeld
        return 3
    else: # sonst
        return 2

Berechnet die nächsten möglichen klickbaren Positionen, falls der Spieler in Phase 1 (Setzphase) ist

In [ ]:
def possible_positions_phase_one(pieces, player):
    return empty_positions(pieces)

Berechnet die nächsten möglichen klickbaren Positionen, falls der Spieler in Phase 2 (Zugphase) ist

In [ ]:
def possible_positions_phase_two(pieces, player):
    collected_piece = (1,2) #ToDo: Implementieren
    neighboring = set(neighboring_positions(collected_piece))
    empty = set(empty_positions(pieces))
    possible_positions = neighboring.intersection(empty)
    #ToDo: Was, wenn kein Stein bewegt werden kann?
    return [possible_positions]

Berechnet die nächsten möglichen klickbaren Positionen, falls der Spieler in Phase 3 (Springphase) ist

In [ ]:
def possible_positions_phase_three(pieces, player):
    return empty_positions(pieces)

Berechnet in welcher Phase sich der Spieler befindet und dementsprechen, wo er als nächstes klicken kann
Kombiniert also die Funktionen oben

In [ ]:
def next_positions(pieces, player):
    phase = get_player_phase(pieces, player)
    if phase == 1:
        return possible_positions_phase_one(pieces, player)
    elif phase == 2:
        return possible_positions_phase_two(pieces, player)
    else:
        return possible_positions_phase_three(pieces, player)

## Mühlen

Vor dem Zug werden die Mühlen gezählt, nach dem Zug werden die Mühlen gezählt. Falls Nachher > Vorher, wurde eine Mühle gebildet. ToDo: Funktioniert so nicht -> Zwickmühle

In [ ]:
def count_muehlen(pieces, player):
    if count_player_pieces(pieces, player) < 3:
        return 0
    #ToDo: Implementieren
    return

## Hilfsfunktionen

In [ ]:
def count_player_pieces(pieces, player):
    [_, board] = pieces
    return [position for ring in board for position in ring].count(player)

In [ ]:
def empty_positions(pieces):
    [_, board] = pieces
    return [(ring, cell) for ring in range(3)
                        for cell in range(8)
                        if  board[ring][cell] == 0 
           ]

In [ ]:
# position = (ring, cell)
def neighboring_positions(position):
    ring, cell = position
    if cell == 0:
        left_neighbor = (ring, 7)
    else:
        left_neighbor = (ring, cell - 1)
    if cell == 7:
        right_neighbor = (ring, 0)
    else:
        right_neighbor = (ring, cell + 1)
        
    positions = [left_neighbor, right_neighbor]
        
    if cell in (1, 3, 5, 7):
        if ring in (0, 2) :
            positions.append((1, cell))
        else:
            positions.append((ring - 1, cell))
            positions.append((ring + 1, cell))
    return positions

In [ ]:
def find_mills():
    global status
    pieces = status.pieces
    player = status.current_player
    oldMills = status.mills
    newMills = set()
    match = set()
    currentPlayerMills = set()

    #find all current mills on the board
    for x in range (0,3):
        for i in [0,2,4,6]:
                if(pieces[1][x][i] == pieces[1][x][i+1] == pieces[1][x][(i+2) % 8] != 0):
                    newMills.add((pieces[1][x][i], x ,i))

    for i in [1,3,5,7]:
        if(pieces[1][0][i] == pieces[1][2][i] == pieces[1][2][i] != 0):
            newMills.add((pieces[1][x][i], 0 ,i))

    #set new mills
    status.mills = newMills   
            

    #if the current mills are the same mills as the old ones or there are now less mills on the board => no new mills
    if(oldMills == newMills or len(oldMills) > len(newMills)):
        return(0)


    #find new mills for current player
    if(oldMills != newMills):
        for mill in newMills:
            for oldMill in oldMills:
                if mill == oldMill:
                    match.add(mill) 
    newMills = newMills.difference(match)

    #obsolete function => pics mills that belong to the current player 
    for (x,y,z) in newMills:
        if x == player:
            currentPlayerMills.add((x,y,z))

    return len(currentPlayerMills) #returns count of new mills for the current player